In [5]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import random
import math
import pandas as pd

In [6]:
def insertar_puntos_intermedios(linea, distancia_min=8.7, distancia_nueva_min=5, distancia_nueva_max=9):
    puntos_nuevos = []
    puntos_originales = list(linea.coords)
    
    for i in range(len(puntos_originales) - 1):
        punto_actual = puntos_originales[i]
        punto_siguiente = puntos_originales[i + 1]
        puntos_nuevos.append(punto_actual)
        
        distancia = Point(punto_actual).distance(Point(punto_siguiente))
        
        while distancia > distancia_min:
            dx = punto_siguiente[0] - punto_actual[0]
            dy = punto_siguiente[1] - punto_actual[1]
            angulo = math.atan2(dy, dx)
            
            distancia_aleatoria = random.uniform(distancia_nueva_min, distancia_nueva_max)
            nuevo_x = punto_actual[0] + math.cos(angulo) * distancia_aleatoria
            nuevo_y = punto_actual[1] + math.sin(angulo) * distancia_aleatoria
            
            punto_actual = (nuevo_x, nuevo_y)
            puntos_nuevos.append(punto_actual)
            
            distancia = Point(punto_actual).distance(Point(punto_siguiente))
        
    puntos_nuevos.append(puntos_originales[-1])
    return LineString(puntos_nuevos)

# Cargar el Shapefile
gdf = gpd.read_file('G:\Ingenio Azucarero Guabira S.A\CITTCA - SEMANAL - Documents\SHP_SEGUIMIENTO\FINAL\BLOQUE.shp')

# Lista para almacenar los GeoDataFrames de cada línea modificada
gdfs_modificados = []

# Iterar sobre cada registro en el GeoDataFrame
for index, row in gdf.iterrows():
    if row.geometry:  # Comprobar si la geometría es None
        linea_modificada = insertar_puntos_intermedios(row.geometry)
        # Crear un GeoDataFrame para la línea modificada y añadir el campo IDD
        gdf_modificado = gpd.GeoDataFrame({'geometry': [linea_modificada], 'IDD': [row['IDD']]}, crs=gdf.crs)
        gdfs_modificados.append(gdf_modificado)

# Concatenar todos los GeoDataFrames en uno solo
gdf_final = pd.concat(gdfs_modificados, ignore_index=True)

# Exportar a Shapefile
gdf_final.to_file("lineas_modificadas.shp")

In [26]:
import geopandas as gpd
import os

# Asumiendo que 'gdf_final' es tu GeoDataFrame resultante
gdf_final = gpd.read_file("lineas_modificadas.shp")

# Obtener una lista de IDD únicos
idd_unicos = gdf_final['IDD'].unique()

# Crear una carpeta principal si no existe
carpeta_principal = "Shapefiles_por_IDD"
os.makedirs(carpeta_principal, exist_ok=True)

# Iterar sobre cada IDD único
for idd in idd_unicos:
    # Seleccionar registros que coincidan con el IDD actual
    registros_seleccionados = gdf_final[gdf_final['IDD'] == idd]
    
    # Crear una subcarpeta para este IDD si no existe
    subcarpeta = os.path.join(carpeta_principal, str(idd))
    os.makedirs(subcarpeta, exist_ok=True)
    
    # Definir el nombre del archivo Shapefile basado en el IDD
    archivo_shp = os.path.join(subcarpeta, f"{idd}.shp")
    
    # Exportar los registros seleccionados a un Shapefile
    registros_seleccionados.to_file(archivo_shp)

print("Shapefiles creados por cada IDD.")


Shapefiles creados por cada IDD.


In [3]:
import geopandas as gpd
import os
import simplekml

def exportar_a_kml(gdf, idd, ruta_carpeta):
    kml = simplekml.Kml()
    for _, fila in gdf.iterrows():
        puntos = fila['geometry'].coords[:]
        linestring = kml.newlinestring(name=str(idd), coords=puntos)
        # Aquí puedes ajustar más propiedades del linestring si es necesario
    kml.save(os.path.join(ruta_carpeta, f"{idd}.kml"))

# Asumiendo que 'gdf_final' es tu GeoDataFrame resultante
gdf_final = gpd.read_file("lineas_modificadas.shp")

# Obtener una lista de IDD únicos
idd_unicos = gdf_final['IDD'].unique()

# Crear una carpeta principal si no existe
carpeta_principal = "KML_por_IDD"
os.makedirs(carpeta_principal, exist_ok=True)

# Iterar sobre cada IDD único
for idd in idd_unicos:
    # Seleccionar registros que coincidan con el IDD actual
    registros_seleccionados = gdf_final[gdf_final['IDD'] == idd]
    
    # Crear una subcarpeta para este IDD si no existe
    subcarpeta = os.path.join(carpeta_principal, str(idd))
    os.makedirs(subcarpeta, exist_ok=True)
    
    # Exportar los registros seleccionados a un archivo KML
    exportar_a_kml(registros_seleccionados, idd, subcarpeta)

print("Archivos KML creados por cada IDD.")


Archivos KML creados por cada IDD.


In [7]:
import geopandas as gpd
import os
import simplekml

def exportar_a_kml(gdf, idd, ruta_carpeta):
    kml = simplekml.Kml()
    for _, fila in gdf.iterrows():
        # Asegurarse de que los puntos están en formato (longitud, latitud) para KML
        puntos = [(x, y) for x, y in zip(fila['geometry'].xy[0], fila['geometry'].xy[1])]
        linestring = kml.newlinestring(name=str(idd))
        linestring.coords = puntos
        # Opciones adicionales de formato pueden ir aquí
    # Guardar el archivo KML
    kml.save(os.path.join(ruta_carpeta, f"{idd}.kml"))

# Cargar el GeoDataFrame y asegurarse de que está en EPSG:4326 para KML
gdf_final = gpd.read_file("lineas_modificadas.shp")

# Transformar el GeoDataFrame a EPSG:4326 si no está ya en ese sistema de coordenadas
if gdf_final.crs != "EPSG:4326":
    gdf_final = gdf_final.to_crs("EPSG:4326")

# Obtener IDD únicos
idd_unicos = gdf_final['IDD'].unique()

# Crear directorio principal si no existe
carpeta_principal = "KML_por_IDD"
os.makedirs(carpeta_principal, exist_ok=True)

for idd in idd_unicos:
    # Seleccionar registros por IDD
    registros_seleccionados = gdf_final[gdf_final['IDD'] == idd]
    
    subcarpeta = os.path.join(carpeta_principal, str(idd))
    os.makedirs(subcarpeta, exist_ok=True)
    
    # Exportar a KML
    exportar_a_kml(registros_seleccionados, idd, subcarpeta)

print("Archivos KML creados por cada IDD.")


Archivos KML creados por cada IDD.
